In [1]:




import os
import pandas as pd
from lxml import etree
from datetime import datetime


ns = {'tei': 'http://www.tei-c.org/ns/1.0', "xml": "http://www.w3.org/XML/1998/namespace"}



#spalte in off oder im ref

def readfiles(orglist, pathtofiles):
    rolenormlist = getrolenormlist()
    orgslist = []

    for filename in os.listdir(pathtofiles):

        regesttree = etree.parse(pathtofiles+"/"+filename)
        regestroot = regesttree.getroot()
        # orgNodes = regestroot.xpath('.//*[contains(@ref,"org__") or contains(@corresp, "org__")]')
        orgNodesref = regestroot.xpath('.//*[contains(@ref,"org__")]')
        orgNodescorresp = regestroot.xpath('.//*[contains(@corresp, "org__") and contains(@type, "off")]')
        orgNodes = orgNodesref + orgNodescorresp
        creationdate = regestroot.find(".//tei:creation/tei:date", ns)
        origindate = regestroot.find(".//tei:history/tei:origin", ns)
        if creationdate is not None:
           date = createdate(creationdate)
        elif origindate is not None:
           date = createdate(origindate)
        else:
           date = " "
        for rs in orgNodes:

            orgdict={}
            text = rs.xpath("normalize-space(.//text())")
            if 'ref' in rs.attrib:
                org= rs.attrib['ref']
                rsfn = rs.xpath("parent::tei:rs[@type='fn']", namespaces=ns)
                if len(rsfn) >0:
                    rsrole = rsfn[0].attrib['role']
                    rsevent = rsfn[0].xpath("parent::tei:rs[@type='event']", namespaces=ns)
                    normed = normrole(rolenormlist, text)
                    if len(rsevent) >0:
                        eventref = rsevent[0].attrib['ref']
                    else: eventref = 'None'
                else:
                    rsrole = 'None'
                    eventref = 'None'
                linking = 'ref'

            elif 'corresp' in rs.attrib:
                org = rs.attrib['corresp']
                rsfn = rs.xpath("ancestor::tei:rs[@type='fn']", namespaces=ns)
                if len(rsfn) > 0:
                    rsrole = rsfn[-1].attrib['role']
                    rsevent = rsfn[-1].xpath("parent::tei:rs[@type='event']", namespaces=ns)
                    normed = normrole(rolenormlist, text)
                    if len(rsevent) >0:
                        eventref = rsevent[-1].attrib['ref']
                    else: eventref = 'None'
                else:
                    rsrole = 'None'
                    eventref = 'None'
                linking = 'corresp'

            orgdict['org'] = org
            orginfos = getorgtype(org, orglist)
            orgdict['type'] = orginfos[0]
            orgdict['observance'] = orginfos[1]
            orgdict['name'] = orginfos[2]
            orgdict['linking'] = linking
            orgdict['date'] = date
            orgdict['role'] = rsrole
            orgdict['event'] = eventref
            orgdict['xml'] = filename
            orgdict['text'] = text
            orgdict['normed'] = normed[1]
            orgslist.append(orgdict)

    writecoloumns(orgslist)


def writecoloumns(orgslist):

    orgsFrame = pd.DataFrame()
    org = []
    mainorg = []
    type = []
    observance =[]
    name = []
    linking =[]
    date = []
    xml = []
    text = []
    role = []
    event = []
    normed = []
    
    for orgdict in orgslist:
        org.append(orgdict['org'])
        mainorg.append("-".join(orgdict['org'].split("-", 2)[:2]))
        type.append(orgdict['type'])
        observance.append(orgdict['observance'])
        name.append(orgdict['name'])
        linking.append(orgdict['linking'])
        date.append(orgdict['date'])
        role.append(orgdict['role'])
        event.append(orgdict['event'])
        xml.append(orgdict['xml'])
        text.append(orgdict['text'])
        normed.append(orgdict['normed'])


    orgsFrame['mainorg'] = mainorg
    orgsFrame['org']= org
    orgsFrame['name_reg | name_orig'] = name
    orgsFrame['type']= type
    orgsFrame['observance'] = observance
    orgsFrame['linking'] = linking
    orgsFrame['role'] = role
    orgsFrame['event'] = event
    orgsFrame['xml'] = xml
    orgsFrame['date'] = date
    orgsFrame['text'] = text
    orgsFrame['text normed'] = normed

    orgsFrame.to_csv("../data/RefCorrespAllOrganisations.csv", sep="\t", encoding="utf-8")



def createdate(node):
    if 'when' in node.attrib:
        when = node.attrib['when'].replace('-','')
        date = datetime.strptime(when, '%Y%m%d').date()
        year = str(date).split("-")[0]
        return date

    elif 'notAfter' in node.attrib:
        notafter = node.attrib['notAfter']
        if "-" in notafter:
            date = datetime.strptime(notafter, '%Y-%m-%d').date()
        else:
            date = datetime.strptime(notafter, '%Y').date()
        year = str(date).split("-")[0]
        return date
    elif 'from' in node.attrib:
        fr =node.attrib['from'].replace('-','')
        to = node.attrib['to'].replace('-','')
        fromdate = datetime.strptime(fr, '%Y%m%d').date()
        todate = datetime.strptime(to, '%Y%m%d').date()
        year = str(fromdate).split("-")[0]
        if fromdate == todate:
            return[fromdate,year]
        else:
            return [fromdate, todate,year]



def getorgtype(ref, orglist):
    ref = ref.replace("#", '')
    org = orglist.find(".//tei:org[@xml:id='"+ref+"']", ns)
    name = " "
    if org is not None:
        regname = org.xpath("normalize-space(./tei:orgName/tei:reg)", namespaces=ns)
        origname = org.xpath("normalize-space(./tei:orgName/tei:orig)", namespaces=ns)
        name = str(regname) + " | " + str(origname)
        if "type" in org.attrib:
            type = org.attrib['type']
            if "Kloster_" in type:
               desc = org.find(".//tei:desc", ns).text
               return [type, desc, name]
            else: return [type, " ", name]
    else: return[" ", " ", " "]

def getrolenormlist():
    role_mapping = pd.read_csv("https://docs.google.com/spreadsheets/d/1_ygvqRnDTg6Rx2wp-RNPvZ3cKQFxfDPo3d7SBDiLO5c/export?gid=0&format=csv", dtype=str)\
    .iloc[:,1:3]
    rolesnormlist = role_mapping.set_index("role").to_dict()

    return rolesnormlist['role_norm']


def normrole(rolenormlist, role):
    if role.lower() in rolenormlist:
        return [role, rolenormlist[role.lower()]]
    elif role in rolenormlist:
        return [role, rolenormlist[role]]
    else:
        return [role, " "]

orglist = etree.parse("../../indices/lists/orgList.xml")
readfiles(orglist, "../../sources/Satzbuch_CD/1448/done")
print("done")





done
